# 本題要將交易資料用月份以及國家進行分組 (\#GroupBy, \#Union )
原題目連結: https://leetcode.com/problems/monthly-transactions-i/description/  

Table: Transactions   
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| country       | varchar |
| state         | enum    |
| amount        | int     |
| trans_date    | date    |
+---------------+---------+
```
id is the primary key of this table.  
The table has information about incoming transactions.  
The state column is an enum of type ["approved", "declined"].  
Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.  
Return the result table in any order.  
The query result format is in the following example.  

範例:  
Ex1:  
Input:   
Transactions table:  
```
+------+---------+----------+--------+------------+
| id   | country | state    | amount | trans_date |
+------+---------+----------+--------+------------+
| 121  | US      | approved | 1000   | 2018-12-18 |
| 122  | US      | declined | 2000   | 2018-12-19 |
| 123  | US      | approved | 2000   | 2019-01-01 |
| 124  | DE      | approved | 2000   | 2019-01-07 |
+------+---------+----------+--------+------------+
```
Output:  
```
+----------+---------+-------------+----------------+--------------------+-----------------------+
| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
+----------+---------+-------------+----------------+--------------------+-----------------------+
| 2018-12  | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01  | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |
+----------+---------+-------------+----------------+--------------------+-----------------------+
```

* 解題想法:  
首先將國家不為null的資料找出來，因為country有可能為null，接著用groupby找出總交易量與通過的交易量，接著找出country為null的資料，並一樣做groupby，最後將兩個table用union all的方式加在一起

# Write your MySQL query statement below
select total.month, total.country, total.trans_count, ifnull(approve.approved_count, 0) as approved_count, total.trans_total_amount, ifnull(approve.approved_total_amount, 0) as approved_total_amount from 
(select DATE_FORMAT(trans_date, '%Y-%m') as month, country, count(*) as trans_count, sum(amount) as trans_total_amount from Transactions where country is not null group by month, country ) total
left join  
(select DATE_FORMAT(trans_date, '%Y-%m') as month, country, count(*) as approved_count, sum(amount) as approved_total_amount from Transactions where state='approved' and country is not null group by month, country) approve
on total.country = approve.country and total.month=approve.month
union all
select total.month, total.country, total.trans_count, ifnull(approve.approved_count, 0) as approved_count, total.trans_total_amount, approve.approved_total_amount from 
(select DATE_FORMAT(trans_date, '%Y-%m') as month, country, count(*) as trans_count, sum(amount) as trans_total_amount from Transactions where country is null group by month, country ) total
left join  
(select DATE_FORMAT(trans_date, '%Y-%m') as month, country, count(*) as approved_count, sum(amount) as approved_total_amount from Transactions where state='approved' and country is null group by month, country) approve
on total.month=approve.month